**Installing and Importing the Required Libraries**

In [2]:
# Installing Libraries
%pip install pyblp


Note: you may need to restart the kernel to use updated packages.


In [3]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyblp
import seaborn as sns

from google.colab import files


ModuleNotFoundError: No module named 'google.colab'

**Loading and Preprocessing the Data**

In [ ]:
# Upload data
uploaded = files.upload()


Saving US Luxury SUVs excluding EVs and NGVs.xlsx to US Luxury SUVs excluding EVs and NGVs (6).xlsx


In [ ]:
# reading data
data = pd.read_excel('US_Luxury_SUVs_excluding_EVs_and_NGVs.xlsx')
subset_data = data.copy()
# Rename columns to remove spaces and special characters
subset_data.columns = subset_data.columns.str.replace(' ', '_').str.replace('.', '').str.replace('(', '').str.replace(')', '')

# Display the first few rows of the DataFrame
data.head()
subset_data.head()


,Model,MSRP_USD,Horsepower,Fuel_Economy_combined,Engine_Type,Transmission,Drivetrain,"Dimensions_L/W/H,_inch",Seating_Capacity,Sales_Q1_2024,Market_Share,Air_Suspension,Panoramic_Sunroof
0,BMW XM,159995,644,14,Twin-Turbocharged (hybrid),8-speed automatic,AWD,201.2/78.9/69.1,5,541,0.001925,no,yes
1,Jeep Grand Wagoneer,93945,510,17,Twin-Turbocharged,8-speed automatic,4WD,214.7/83.6/75.6,7,3550,0.012634,yes,yes
2,Lexus LX,93915,409,19,Twin-Turbocharged,10-speed automatic,4WD,200.6/78.3/74.2,5,1697,0.006040,yes,no
3,Lincoln Navigator,85260,440,18,Twin-Turbocharged,10-speed automatic,4WD,210/79.9/76.3,7,3127,0.011129,no,yes
4,Lexus GX,64250,349,17,Twin-Turbocharged,10-speed automatic,4WD,197.1/83.2/75.6,7,3858,0.013731,no,no


In [ ]:
# Creating the AWD dummy variable: 1 if AWD or 4WD, 0 otherwise
subset_data["AWD_dummy"] = np.where(subset_data["Drivetrain"].isin(["AWD", "4WD"]), 1, 0)

# Creating the Air Suspension dummy variable: 1 if Yes, 0 if No
subset_data["Air_Suspension_dummy"] = np.where(subset_data["Air_Suspension"] == "yes", 1, 0)

# Creating the Panoramic Sunroof dummy variable: 1 if Yes, 0 if No
subset_data["Panoramic_Sunroof_dummy"] = np.where(subset_data["Panoramic_Sunroof"] == "yes", 1, 0)


In [ ]:
# Select a limited number of variables for the initial estimation
variables = ['MSRP_USD', 'Horsepower', 'Fuel_Economy_combined', 'AWD_dummy', 'Air_Suspension_dummy', 'Panoramic_Sunroof_dummy']

# Drop rows with missing values in the selected columns
subset_data = subset_data.dropna(subset=variables + ['Sales_Q1_2024', 'Market_Share'])


In [ ]:
# Define the product characteristics (X) and the instruments (Z)
# Prepare the data for the BLP model
product_data_blp  = subset_data[["MSRP_USD", "Horsepower", "Fuel_Economy_combined", "AWD_dummy", "Air_Suspension_dummy", "Panoramic_Sunroof_dummy"]]

print(product_data_blp.dtypes)
print(product_data_blp.head())

# Market data
market_data = {
    'product_ids': data.index,
    'market_ids': np.ones(subset_data.shape[0]),
    'shares': subset_data['Market_Share']
}

# Combine the data into the format required by pyblp
# X1: Linear Characteristics
X1_formulation = pyblp.Formulation('1 + MSRP_USD + horsepower + Fuel_Economy_combined + AWD_dummy + Air_Suspension_dummy + Panoramic_Sunroof_dummy')
# X2: Nonlinear Characteristics (using the same variables here for simplicity)
X2_formulation = pyblp.Formulation('1 + horsepower + Fuel_Economy_combined + AWD_dummy + Air_Suspension_dummy + Panoramic_Sunroof_dummy')
# X3: Log Cost Characteristics
X3_formulation = pyblp.Formulation('1 +log(horsepower) + log(Fuel_Economy_combined) + AWD_dummy')


MSRP_USD                   int64
Horsepower                 int64
Fuel_Economy_combined      int64
AWD_dummy                  int64
Air_Suspension_dummy       int64
Panoramic_Sunroof_dummy    int64
dtype: object
   MSRP_USD  Horsepower  Fuel_Economy_combined  AWD_dummy  \
0    159995         644                     14          1   
1     93945         510                     17          1   
2     93915         409                     19          1   
3     85260         440                     18          1   
4     64250         349                     17          1   

   Air_Suspension_dummy  Panoramic_Sunroof_dummy  
0                     0                        1  
1                     1                        1  
2                     1                        0  
3                     0                        1  
4                     0                        0  


In [4]:
product_formulations = (X1_formulation, X2_formulation, X3_formulation)


NameError: name 'X1_formulation' is not defined

In [ ]:


# Create the problem instance with integration method
mc_integration = pyblp.Integration('monte_carlo', size=200, specification_options={'seed': 0})
problem = pyblp.Problem(product_formulations, market_data, product_data_blp, integration=mc_integration)

# Define initial values
initial_sigma = np.diag([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
initial_beta = np.array([1.0, -1.0, 1.0, 1.0, 1.0, 1.0])
initial_gamma = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

# Set up optimization routine
tnc = pyblp.Optimization('tnc', {'maxfun': 500})

# Solve the problem
result = problem.solve(initial_sigma, initial_beta, initial_gamma, tnc)


Initializing the problem ...


PatsyError: Failed to load data for 'Air_Suspension_dummy' because of the above exception.
    1 + MSRP_USD + horsepower + Fuel_Economy_combined + AWD_dummy + Air_Suspension_dummy + Panoramic_Sunroof_dummy
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
print(product_data_blp.columns)


Index(['Horsepower', 'Fuel_Economy_combined', 'AWD_dummy'], dtype='object')


In [ ]:
# Print Column Names: Ensure that 'Fuel_Economy_combined' is present in product_data_blp.
print(product_data_blp.columns)


NameError: name 'product_data_blp' is not defined